In [23]:
import dask.array as da

In [24]:
data=da.random.randint(2,size=(32000,51,51,51),chunks=(100,51,51,51))

In [25]:
data

,Array,Chunk
Bytes,33.96 GB,106.12 MB
Shape,"(32000, 51, 51, 51)","(100, 51, 51, 51)"
Count,320 Tasks,320 Chunks
Type,int64,numpy.ndarray


In [26]:
import time

In [27]:
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation,FlattenTransformer
from dask_ml.decomposition import PCA
from sklearn.pipeline import Pipeline

In [28]:
from dask.distributed import Client, progress


In [29]:
client = Client()
client

/nix/store/02z0vx25y0x2arfvcp89pvy52a3q77wf-python3.7-distributed-2.9.1/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:37465 Dashboard: http://127.0.0.1:35787/status,Cluster Workers: 4 Cores: 8 Memory: 33.59 GB


In [30]:
bl1 = Pipeline(steps=[
    ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)])),
    ('flatten', FlattenTransformer()),  
    ('reducer',PCA(n_components=3))])


In [32]:
tic = time.time()
a=bl1.fit(data)
toc = time.time()
print(tic-toc)

-181.32060265541077


In [33]:
a

Pipeline(memory=None,
         steps=[('discritize',
                 PrimitiveTransformer(chunks=None, max_=1.0, min_=0.0,
                                      n_state=2)),
                ('Correlations',
                 TwoPointCorrelation(correlations=[(1, 1)], cutoff=15,
                                     periodic_boundary=True)),
                ('flatten', FlattenTransformer()),
                ('reducer',
                 PCA(copy=True, iterated_power=0, n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False))],
         verbose=False)

In [17]:
# a.visualize()

In [35]:
tic = time.time()
b=bl1.transform(data).compute()
toc = time.time()
print(tic-toc)
print(b)

-178.6685643196106
[[ 1.57394951e-01 -8.00667362e-04 -1.42588811e-03]
 [-3.21063384e-01 -1.17210664e-03 -2.71622314e-03]
 [-1.30771410e-01  1.23761756e-03  7.08329126e-05]
 ...
 [-6.38604852e-02  6.79004171e-05 -3.28728850e-03]
 [-4.00808520e-01 -6.56761555e-04  8.69151375e-04]
 [-9.63854845e-02 -1.63763365e-04 -3.03665581e-03]]
